# Import necessary libraries and modules and creating the Json Encoder class

In [1]:
from Services.JSONHandler import JSONHandler
from Services.SearchEngine import SearchEngine
from Services.SearchEvaluator import SearchEvaluator
import json

from bson import ObjectId
from tqdm import tqdm  


import fasttext
ft_model = fasttext.load_model('./Models/antique_model.bin')
class JSONEncoder(json.JSONEncoder):
    """ Extend json-encoder class to handle ObjectId types. """
    def default(self, obj):
        if isinstance(obj, ObjectId):
            return str(obj)
        return super().default(obj)

In [3]:
from Services.MilvusDB import MilvusDB

MilvusDB.connect()

# Define file paths
dataset_path = './DataSets/Antique/JSON/Antique_Docs.json'
index_path = './DataSets/Antique/inverted-index.json'
tfidf_file = "./DataSets/Antique/tfidf-matrix.npz"
vectorizer_file = "./DataSets/Antique/vectorizer.pkl"
doc_ids_file_path = "./DataSets/Antique/doc_ids.json"
qrels_path = './DataSets/Antique/JSON/Antique_Qrels.json'

# Load qrels

qrels = SearchEvaluator.load_qrels(qrels_path)

# Load queries
queries = JSONHandler.convert_from_json('./DataSets/Antique/JSON/Antique_Queries.json')

# Initialize results dictionary
results_dict = {}
mrr_antique_sum = 0
map_antique_sum = 0
recall_antique_sum = 0
p10_antique_sum = 0

# Process each query and evaluate
for query in tqdm(queries, desc="Processing Queries"):
    query_id = query['query_id']
    query_text = query['query']
    search_results = SearchEngine.search_with_embedding(query_text, "antique", ft_model,100)
    results = SearchEvaluator.evaluate_search_engine_with_embedding(query_id, search_results, qrels)
    results_dict[query_id] = results
    mrr_antique_sum+=results["MRR"]
    map_antique_sum+=results["MAP"]
    recall_antique_sum+=results['R']
    p10_antique_sum+=results['P@10']

mrr_antique_avg = mrr_antique_sum/len(queries)
map_antique_avg = map_antique_sum/len(queries)
p10_antique_avg = p10_antique_sum/len(queries)
recall_antique_avg = recall_antique_sum/len(queries)

print(f'AVG MRR : {mrr_antique_avg} ')
print(f'AVG MAP : {map_antique_avg} ')
print(f'AVG P@10 : {p10_antique_avg} ')
print(f'AVG R : {recall_antique_avg} ')

Connected to Milvus at localhost:19530


Processing Queries:   0%|          | 0/2426 [00:00<?, ?it/s]

Processing Queries: 100%|██████████| 2426/2426 [01:19<00:00, 30.35it/s]

AVG MRR : 0.5691078135018778 
AVG MAP : 0.5216815389629577 
AVG P@10 : 0.3816982687551541 
AVG R : 0.7054925004131835 
